In [1]:
import numpy as np

_frequencies = [1265, 897, 643, 1190, 521, 1688, 778, 1999, 1690, 1433, 1796, 1266, 1046, 1353]
frequencies = np.array(_frequencies) / np.array(_frequencies).sum()

frequencies

array([0.07201822, 0.05106746, 0.03660689, 0.06774836, 0.02966126,
       0.0961002 , 0.04429263, 0.11380586, 0.09621406, 0.08158269,
       0.10224879, 0.07207515, 0.05955024, 0.07702818])

In [ ]:
a = TiDBHypo("imdbload")
# b = a.execute_create_hypo("aka_name#surname_pcode")
b = a.get_hypo_indexes_from_one_table("aka_name")
b
# a.test()


In [ ]:

string = "KEY `hypo_aka_name_name_pcode_nf_idx` (`name_pcode_nf`) /* HYPO INDEX */"

if "HYPO INDEX" in string:
    tmp = string.split("`")
    idx_name = tmp[1]
    table_name = "aka_name"
    hypo = f"{table_name}.{idx_name}"
    print(hypo)
    
    

In [ ]:
import numpy as np

np.zeros(60, dtype=np.cfloat)

In [ ]:
import pymysql


def get_hypo_indexes_from_table(conn, table_name):
    statement = f"show create table {table_name}"
    cur = conn.cursor()
    result = cur.execute(statement)
    hypo_indexes = []
    for line in result[1].split("\n"):
        if "HYPO INDEX" in line:
            tmp = line.split("`")
            hypo_indexes.append(tmp[1])
    return hypo_indexes
    
    

conn = pymysql.connect(host='127.0.0.1',
                     port=4000,
                     user='root',
                     password='',
                     database="imdbload",
                     local_infile=True)

cur = conn.cursor()



statement = "create index hypo_aka_name_name_pcode_nf_idx type hypo on aka_name (name_pcode_nf);"
s2 = "show create table aka_name;"


cur.execute(statement)

print(get_hypo_indexes_from_table(conn, "aka_name"))

cur.execute(s2)

print(cur.fetchall())

In [2]:
indexes = [1.0, 0.0, 1.0]
index_candidates=["123","423123", "23123"]

selected_indexes = [index_candidates[i] for i, idx in enumerate(indexes) if idx == 1.0]

selected_indexes

['123', '23123']

In [9]:
import numpy as np

action = np.random.randint(0, 65)



a = np.float64(0)
a

0.0

In [7]:
a = np.zeros(10)
a



array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])